In [ ]:
library(corrplot)
library(ggplot2)
library(tidyr)
library(dplyr)
library(purrr)
library(GGally)
library(MASS)
library(caret)

We'll analyze the Kaggle House Prices dataset and predict the prices of houses.

In [ ]:
train_data <- read.csv("data/train.csv")
test_data <- read.csv("data/test.csv")

Let's start by taking at look at which variables are available in the dataset.

In [ ]:
str(train_data)

We have a mix of numerical and categorical variables with missing values. Let's take a look at the sale price variable, which is the target variable.

In [ ]:
qplot(train_data$SalePrice)

It's clear that the distribution of SalePrice has a positive skew and is not exactly normal.

In [ ]:
options(repr.plot.width=20, repr.plot.height=20)
nums <- unlist(lapply(train_data, is.numeric), use.names = FALSE)
train_data %>% select_if(is.numeric) %>%  gather(cols, value) %>%  ggplot(aes(value)) + geom_histogram() + facet_wrap(~cols, scales='free')
options(repr.plot.width=12, repr.plot.height=12)

Well, many variables are not normally distributed. Also, many of them seem to be count variables with discrete values. Some have very strong skew and kurtosis, while others have zero-inflated distributions. These facts are important to understand before fitting a model.

Let's also have a look at correlation matrix plots to see if we can spot some obvious or interesting correlations. We'll use Spearman correlation since most variables are not normally distributed.

In [ ]:
train_data %>% select_if(is.numeric) %>% cor(use = "complete.obs", method = "spearman") -> correlations
corrplot(correlations, method = "circle")

Some variables seem to provide little extra information over others, for example, YearBuilt and GarageYrBlt. This means there is quite a lot of multicolinearity in the data. This is very relevant if we want to fit a linear model.

Let's finally take a look at the variables with the highest correlation with the target variable.

In [ ]:
correlations[, 38] %>% sort(decreasing = TRUE) %>% print()

Overall quality, ground living area, year built, garage capacity and the number of full bathrooms are the variables most correlated with the target variable.

Let's take a look at the number of levels of each categorical variable. Variables with only two levels are transformed to numerical variables.

In [ ]:
train_data <- as.data.frame(unclass(train_data), stringsAsFactors = TRUE)

train_data$Street <- as.numeric(train_data$Street)
train_data$Alley <- as.numeric(train_data$Alley)
train_data$Utilities <- as.numeric(train_data$Utilities)
train_data$CentralAir <- as.numeric(train_data$CentralAir)


# test data
test_data <- as.data.frame(unclass(test_data), stringsAsFactors = TRUE)

test_data$Street <- as.numeric(test_data$Street)
test_data$Alley <- as.numeric(test_data$Alley)
test_data$Utilities <- as.numeric(test_data$Utilities)
test_data$CentralAir <- as.numeric(test_data$CentralAir)

In [ ]:
lapply(train_data, is.na) %>% sapply(sum) %>% sapply(function(x) x / 1460) %>% sort(decreasing = TRUE) -> na_proportion

print(na_proportion[0:10])

We'll simply remove the variables with more than 20% of missing values.

In [ ]:
train_data_cleaned <- subset(train_data, select = -c(PoolQC, MiscFeature, Alley, Fence, FireplaceQu))

test_data_cleaned <- subset(test_data, select = -c(PoolQC, MiscFeature, Alley, Fence, FireplaceQu))

Now, let's take a look at the variables with the highest correlation with the target variable.

In [ ]:
correlations[, 38] %>% sort(decreasing = TRUE) %>% names() -> cols
train_data_cleaned %>% dplyr::select(cols[0:10]) %>% ggpairs(aes(alpha = 0.75)) + theme_bw()

Looking at the first column, we can see that in fact those variables seem to be positively correlated with the target variable. However, only overall quality seems to be reasonably normally distributed. Let's take a look at their distributions and transform them to get more normal-like distributions.

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)

qqnorm(train_data_cleaned$SalePrice)
qqline(train_data_cleaned$SalePrice)

train_data_cleaned$SalePriceLog <- log(train_data_cleaned$SalePrice)
qqnorm(train_data_cleaned$SalePriceLog)
qqline(train_data_cleaned$SalePriceLog)

In [ ]:
qqnorm(train_data_cleaned$GrLivArea)
qqline(train_data_cleaned$GrLivArea)

train_data_cleaned$GrLivAreaLog <- log(train_data_cleaned$GrLivArea)
test_data_cleaned$GrLivAreaLog <- log(test_data_cleaned$GrLivArea)
qqnorm(train_data_cleaned$GrLivAreaLog)
qqline(train_data_cleaned$GrLivAreaLog)

In [ ]:
qqnorm(train_data_cleaned$YearBuilt)
qqline(train_data_cleaned$YearBuilt)

train_data_cleaned$YearBuiltLogInv <- log(1 / train_data_cleaned$YearBuilt)
test_data_cleaned$YearBuiltLogInv <- log(1 / test_data_cleaned$YearBuilt)
qqnorm(train_data_cleaned$YearBuiltLogInv)
qqline(train_data_cleaned$YearBuiltLogInv)

This one is so skewed the transformation is not enough to make it normal.

In [ ]:
qqnorm(train_data_cleaned$GarageArea)
qqline(train_data_cleaned$GarageArea)

train_data_cleaned$GarageAreaLog <- log(train_data_cleaned$GarageArea + 1)
test_data_cleaned$GarageAreaLog <- log(test_data_cleaned$GarageArea + 1)
qqnorm(train_data_cleaned$GarageAreaLog)
qqline(train_data_cleaned$GarageAreaLog)

Since this one has a lot of zeros, we'll create a new variable indicating if a garage is present or not. We'll also turn the zero values into NAs.

In [ ]:
train_data_cleaned$GaragePresent <- as.numeric(train_data_cleaned$GarageArea > 0)
train_data_cleaned$GarageAreaLog <- ifelse(train_data_cleaned$GaragePresent, train_data_cleaned$GarageAreaLog, NA)
summary(train_data_cleaned$GarageAreaLog)
summary(train_data_cleaned$GaragePresent)

test_data_cleaned$GaragePresent <- as.numeric(test_data_cleaned$GarageArea > 0)
test_data_cleaned$GarageAreaLog <- ifelse(test_data_cleaned$GaragePresent, train_data_cleaned$GarageAreaLog, NA)

In [ ]:
qqnorm(train_data_cleaned$GarageYrBlt)
qqline(train_data_cleaned$GarageYrBlt)

train_data_cleaned$GarageYrBltLogInv <- log(1 / train_data_cleaned$GarageYrBlt)
test_data_cleaned$GarageYrBltLogInv <- log(1 / test_data_cleaned$GarageYrBlt)
qqnorm(train_data_cleaned$GarageYrBltLogInv)
qqline(train_data_cleaned$GarageYrBltLogInv)

In [ ]:
qqnorm(train_data_cleaned$TotalBsmtSF)
qqline(train_data_cleaned$TotalBsmtSF)

train_data_cleaned$TotalBsmtSFLog <- log(1 + train_data_cleaned$TotalBsmtSF)
test_data_cleaned$TotalBsmtSFLog <- log(1 + test_data_cleaned$TotalBsmtSF)
qqnorm(train_data_cleaned$TotalBsmtSFLog)
qqline(train_data_cleaned$TotalBsmtSFLog)

Again, it's useful to create a new variable indicating if a basement is present or not and turn zeros into NAs.

In [ ]:
train_data_cleaned$BasementPresent <- as.numeric(train_data_cleaned$TotalBsmtSF > 0)
train_data_cleaned$TotalBsmtSFLog <- ifelse(train_data_cleaned$BasementPresent, train_data_cleaned$TotalBsmtSFLog, NA)
summary(train_data_cleaned$TotalBsmtSFLog)
summary(train_data_cleaned$BasementPresent)

test_data_cleaned$BasementPresent <- as.numeric(test_data_cleaned$TotalBsmtSF > 0)
test_data_cleaned$TotalBsmtSFLog <- ifelse(test_data_cleaned$BasementPresent, train_data_cleaned$TotalBsmtSFLog, NA)

In [ ]:
print(cols)

qplot(train_data_cleaned$YearRemodAdd, train_data_cleaned$SalePrice, geom = "point")
qqnorm(train_data_cleaned$YearRemodAdd)
qqline(train_data_cleaned$YearRemodAdd)

This is again too skewed and there's also a suspicious excess of 1950s in this variable. For this reason, We'll binarize this variable around 1985 (by visual inspection).

In [ ]:
train_data_cleaned$YearRemodAddBinary <- as.numeric(train_data_cleaned$YearRemodAdd >= 1985)
summary(train_data_cleaned$YearRemodAddBinary)

test_data_cleaned$YearRemodAddBinary <- as.numeric(test_data_cleaned$YearRemodAdd >= 1985)

In [ ]:
qqnorm(train_data_cleaned$X1stFlrSF)
qqline(train_data_cleaned$X1stFlrSF)

train_data_cleaned$X1stFlrSFLog <- log(train_data_cleaned$X1stFlrSF)
test_data_cleaned$X1stFlrSFLog <- log(test_data_cleaned$X1stFlrSF)
qqnorm(train_data_cleaned$X1stFlrSFLog)
qqline(train_data_cleaned$X1stFlrSFLog)

Perfect, now we have a data table with transformed variables and some new ones for those that are most correlated with the target variable. Let's visualize again the correlation patterns between the target variables and the transformed variables.

In [ ]:
options(repr.plot.width=12, repr.plot.height=12)

transformed_cols = c(
    "SalePriceLog", "OverallQual", "GrLivAreaLog",
    "YearBuiltLogInv", "GarageCars", "FullBath",
    "GarageAreaLog", "GarageYrBltLogInv", "TotalBsmtSFLog",
    "YearRemodAddBinary", "X1stFlrSFLog")

train_data_cleaned %>% dplyr::select(transformed_cols) %>% ggpairs(aes(alpha = 0.75)) + theme_bw()

The first column looks better now with the transformed variables. Let's split the data into train and validation sets and then fit a simple linear regression model.

In [ ]:
set.seed(42)
split_index <- caret::createDataPartition(train_data_cleaned$SalePriceLog, p = 0.8, list = FALSE)
train_data_cleaned_train <- train_data_cleaned[split_index,]
train_data_cleaned_val <- train_data_cleaned[-split_index,]

In [ ]:
formula <- paste("SalePrice ~ ", paste(transformed_cols, collapse = " + "), sep = "")

linear_model <- lm(formula, data = train_data_cleaned_train)

summary(linear_model)

Let's calculate the root mean squared error, which is the metric used in this competition.

In [ ]:
(predict(linear_model, train_data_cleaned_val) - train_data_cleaned_val$SalePriceLog) %>% mean(na.rm = TRUE) %>% sqrt() -> lm_rmse
print(lm_rmse)

From the correlation plots we can see there's quite some multicolinearity happening here. Still, the model doesn't seem all that bad with R^2 = 0.91. Let's add the categorical variables to the model.

In [ ]:
train_data_cleaned_train %>% select_if(purrr::negate(is.numeric)) %>% names() -> categorical_cols

lm_cols = c(transformed_cols, categorical_cols)

formula <- paste("SalePrice ~ ", paste(lm_cols, collapse = " + "), sep = "")

linear_model <- lm(formula, data = train_data_cleaned_train)

summary(linear_model)

In [ ]:
train_data_complete <- tidyr::drop_na(train_data_cleaned_train)
val_data_complete <- tidyr::drop_na(train_data_cleaned_val)

(predict(linear_model, train_data_complete) - train_data_complete$SalePriceLog) %>% mean(na.rm = TRUE) %>% sqrt() -> lm_rmse
print(lm_rmse)

Here train data is used for simplicity since there are many categorical variables whose levels are not the same in both train and validation data.

R² is now 0.95. However, we should take this with a grain of salt since there are far too many categorical variables and levels and too much multicolinearity. It's a good idea to either use a form of variable selection such as lasso or another type of model that can implicitly handle the excess of variables.